<a href="https://colab.research.google.com/github/bitanb1999/TalentSumoAI/blob/main/Speech_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Speech-to-text

In [1]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 198 kB/s 


In [3]:
!pip install pydub

In [4]:
from os import path
from pydub import AudioSegment

# files                                                                         
src = "/content/20220326-202223.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

<_io.BufferedRandom name='test.wav'>

In [5]:
#import library
import speech_recognition as sr

# Initialize recognizer class (for recognizing the speech)
r = sr.Recognizer()

# Reading Audio file as source
# listening the audio file and store in audio_text variable

with sr.AudioFile('/content/test.wav') as source:
    
    audio_text = r.listen(source)
    
# recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
    try:
        
        # using google speech recognition
        text = r.recognize_google(audio_text)
        print('Converting audio transcripts into text ...')
        print(text)
     
    except:
         print('Sorry.. run again...')

Converting audio transcripts into text ...
My Name Is Earl


In [7]:
# importing libraries
import speech_recognition as sr

import os

from pydub import AudioSegment
from pydub.silence import split_on_silence

# a function that splits the audio file into chunks
# and applies speech recognition
def silence_based_conversion(path = "/content/test.wav"):

	# open the audio file stored in
	# the local system as a wav file.
	song = AudioSegment.from_wav(path)

	# open a file where we will concatenate
	# and store the recognized text
	fh = open("recognized.txt", "w+")
		
	# split track where silence is 0.5 seconds
	# or more and get chunks
	chunks = split_on_silence(song,
		# must be silent for at least 0.5 seconds
		# or 500 ms. adjust this value based on user
		# requirement. if the speaker stays silent for
		# longer, increase this value. else, decrease it.
		min_silence_len = 500,

		# consider it silent if quieter than -16 dBFS
		# adjust this per requirement
		silence_thresh = -16
	)

	# create a directory to store the audio chunks.
	try:
		os.mkdir('audio_chunks')
	except(FileExistsError):
		pass

	# move into the directory to
	# store the audio files.
	os.chdir('audio_chunks')

	i = 0
	# process each chunk
	for chunk in chunks:
			
		# Create 0.5 seconds silence chunk
		chunk_silent = AudioSegment.silent(duration = 10)

		# add 0.5 sec silence to beginning and
		# end of audio chunk. This is done so that
		# it doesn't seem abruptly sliced.
		audio_chunk = chunk_silent + chunk + chunk_silent

		# export audio chunk and save it in
		# the current directory.
		print("saving chunk{0}.wav".format(i))
		# specify the bitrate to be 192 k
		audio_chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav")

		# the name of the newly created chunk
		filename = 'chunk'+str(i)+'.wav'

		print("Processing chunk "+str(i))

		# get the name of the newly created chunk
		# in the AUDIO_FILE variable for later use.
		file = filename

		# create a speech recognition object
		r = sr.Recognizer()

		# recognize the chunk
		with sr.AudioFile(file) as source:
			# remove this if it is not working
			# correctly.
			r.adjust_for_ambient_noise(source)
			audio_listened = r.listen(source)

		try:
			# try converting it to text
			rec = r.recognize_google(audio_listened)
			# write the output to the file.
			fh.write(rec+". ")

		# catch any errors.
		except sr.UnknownValueError:
			print("Could not understand audio")

		except sr.RequestError as e:
			print("Could not request results. check your internet connection")

		i += 1

	os.chdir('..')


if __name__ == '__main__':
		
	print('Enter the audio file path')

	path = input()

	silence_based_conversion(path)


Enter the audio file path
/content/test.wav


In [16]:
def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.
    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source) # #  analyze the audio source for 1 second
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #   update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable/unresponsive"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response

###Link: https://realpython.com/python-speech-recognition/